# Submission 2: Denoising Autoencoders
Trains CNN autoencoders to denoise pneumonia images based on [Medical image denoising using convolutional denoising autoencoders](https://arxiv.org/pdf/1608.04667.pdf). Contains 3 CNN autoencoders:
1. Autoencoder for denoising noisy images with lambda = 25
2. Autoencoder for denoising noisy images with lambda = 50
3. Autoencoder for denoising noisy images with lambda = 75

### Import Libraries

In [35]:
from sklearn.model_selection import train_test_split
import os
from torchvision import datasets
from torchvision.transforms import ToTensor

### Load Dataset

In [36]:
original_ds = r"datasets/pneumonia"
original_ds_filenames = os.listdir(original_dataset)

In [37]:
# Partition data into 80% training and 20% test data
train_filenames, test_filenames = train_test_split(original_ds_filenames, train_size=0.8, random_state=0)

### Generating Noisy Images

In [38]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [42]:
train_features, train_labels = next(iter(dataloader))
print(train_features[0])

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.4392, 0.0000, 0.0000, 0.0039, 0.0000,
          0.0549, 0.2941, 0.0000, 0.0039, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.3804, 0.0000, 0.0000, 0.0118, 0.0000,
          0.3176, 0.4745, 0.0000, 0.0078, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.2039, 0.0196, 0.0000, 0.0000, 0.0000,
          0.0392, 0.0078, 0.0000, 0.0039, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0039, 0.0000, 0.0863, 0.4000, 0.1216, 0.3451, 0.0078, 0.1961,
          0.0549, 0.1961, 0.0000, 0.0000, 0.0000, 0.0000,

In [ ]:
# Create 3 sets of noisy images based on the original dataset

In [ ]:
# Function to create a noisy copy of the original dataset with the given lambda
def generate_noisy_dataset(lambda_val):
    for filename in os.listdir(original_dataset):
        

## Evaluating results

**Why is poisson distribution the ideal one to use to simulate noise for medical images? Why not gaussian or something else? Answer in terms of relevance of medical applications.**

{Answer}

**Which one performed the best? Why do you think this is the case?**

{Answer}